In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 


# Q1

In [13]:
import torch.nn.functional as F
image = torch.rand(6,6)
for i in range(2):
    image = image.unsqueeze(dim=0)
print("Image shape: ",image.shape)

kernel = torch.rand(3,3)
for i in range(2):
    kernel = kernel.unsqueeze(dim=0)
print("Kernel : ",kernel.shape)
# General Parameters
output = F.conv2d(image,kernel,stride=1,padding=0)
print("Output Image: ",output.shape)
# Changing Padding Parameter
output = F.conv2d(image,kernel,stride=1,padding=1)
print("Output Image-2: ",output.shape)
# 




Image shape:  torch.Size([1, 1, 6, 6])
Kernel :  torch.Size([1, 1, 3, 3])
Output Image:  torch.Size([1, 1, 4, 4])
Output Image-2:  torch.Size([1, 1, 6, 6])


# Q2

In [19]:
import torch.nn as nn
image = torch.rand(6,6)
for i in range(2):
    image = image.unsqueeze(dim=0)
print("Image shape: ",image.shape)
conv = nn.Conv2d(1,3,kernel_size=3,padding=1,bias = False)
res1 = conv(image)
weights = conv.weight
print(weights.shape)
res2 = F.conv2d(image,weights,padding = 1, bias = None)
print(res1.shape)
print(res2.shape)

Image shape:  torch.Size([1, 1, 6, 6])
torch.Size([3, 1, 3, 3])
torch.Size([1, 3, 6, 6])
torch.Size([1, 3, 6, 6])


# Q3

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch.optim as optimizer 

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_dataset = datasets.MNIST(root='./data',train = True, transform = transform)
test_dataset = datasets.MNIST(root='./data',train = False, transform = transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class CNNMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(64,32)
        self.output = nn.Linear(32,10)
        self.maxpool = nn.MaxPool2d((2,2),stride=2)
    def forward(self,x):
        # Convolution
        x = nn.Conv2d(1,64,kernel_size=3)(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = nn.Conv2d(64,128,kernel_size=3)(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = nn.Conv2d(128,64,kernel_size=3)(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # Classification
        x = self.fc1(x.view(64,-1))
        x = self.relu(x)

        x = self.output(x)
        x = self.softmax(x)
        return x

model = CNNMnist().to(device)
criterion = nn.CrossEntropyLoss()
opt = optimizer.Adam(model.parameters(),lr = 0.001)
epochs = 20

for epoch in range(epochs):
    for images, labels in train_loader:
        images = images.to(device)
        lables = labels.to(device)
        opt.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()
    print("Epoch {epoch} LOSS: {loss}")





Using device: cuda


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [4]:
import torch
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")


CUDA Available: True


In [9]:
import torch
import torch.nn as nn
import torch.optim as optimizer
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transforms remain the same
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class CNNMnist(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolution layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        
        # Pooling and activation
        self.maxpool = nn.MaxPool2d((2,2), stride=2)
        self.relu = nn.ReLU()
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 3 * 3, 32)  # Adjusted input size based on convolution output
        self.output = nn.Linear(32, 10)
        
        # Softmax
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Convolution layers
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x

model = CNNMnist().to(device)
criterion = nn.CrossEntropyLoss()
opt = optimizer.Adam(model.parameters(), lr=0.0008)

# Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for images, labels in train_loader:
        # Move data to CUDA device
        images = images.to(device)
        labels = labels.to(device)
        
        opt.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Optional: Evaluation on test set
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Run evaluation
evaluate_model(model, test_loader, device)
torch.save(model.state_dict(),"MNIST_model.pt")


Using device: cuda
Epoch 1/20, Loss: 1.7208
Epoch 2/20, Loss: 1.6187
Epoch 3/20, Loss: 1.5716
Epoch 4/20, Loss: 1.5684
Epoch 5/20, Loss: 1.5584
Epoch 6/20, Loss: 1.4754
Epoch 7/20, Loss: 1.4728
Epoch 8/20, Loss: 1.4712
Epoch 9/20, Loss: 1.4709
Epoch 10/20, Loss: 1.4715
Epoch 11/20, Loss: 1.4700
Epoch 12/20, Loss: 1.4694
Epoch 13/20, Loss: 1.4699
Epoch 14/20, Loss: 1.4700
Epoch 15/20, Loss: 1.4689
Epoch 16/20, Loss: 1.4684
Epoch 17/20, Loss: 1.4694
Epoch 18/20, Loss: 1.4688
Epoch 19/20, Loss: 1.4689
Epoch 20/20, Loss: 1.4681
Test Accuracy: 99.01%


In [10]:
for x in model.state_dict().keys():
    print("x value : ",x,'\t',"size : ",model.state_dict()[x].size())

x value :  conv1.weight 	 size :  torch.Size([64, 1, 3, 3])
x value :  conv1.bias 	 size :  torch.Size([64])
x value :  conv2.weight 	 size :  torch.Size([128, 64, 3, 3])
x value :  conv2.bias 	 size :  torch.Size([128])
x value :  conv3.weight 	 size :  torch.Size([64, 128, 3, 3])
x value :  conv3.bias 	 size :  torch.Size([64])
x value :  fc1.weight 	 size :  torch.Size([32, 576])
x value :  fc1.bias 	 size :  torch.Size([32])
x value :  output.weight 	 size :  torch.Size([10, 32])
x value :  output.bias 	 size :  torch.Size([10])


In [ ]:
config = {"Small kernel":{
                'kernel_size' : [3,3,3],
                'base_channels': 32,
                'fc1':32
                },
            "Mid kernel":{
                'kernel_size' : [3,3,3],
                'base_channels': 16,
                'fc1':30
            }
        }
class CNNMnist(nn.Module):
    def __init__(self,config):
        super().__init__()
        # Convolution layers
        channels = config['base_channels']
        fc1_neurons = config['fc1']
        self.conv1 = nn.Conv2d(1, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, 2*channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*channels, channels, kernel_size=3, padding=1)
        
        # Pooling and activation
        self.maxpool = nn.MaxPool2d((2,2), stride=2)
        self.relu = nn.ReLU()
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 3 * 3, fc1_neurons)  # Adjusted input size based on convolution output
        self.output = nn.Linear(fc1_neurons, 10)
        
        # Softmax
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Convolution layers
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x



model = CNNMnist().to(device)
criterion = nn.CrossEntropyLoss()
opt = optimizer.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for images, labels in train_loader:
        # Move data to CUDA device
        images = images.to(device)
        labels = labels.to(device)
        
        opt.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

